# Задача

Необходимо предсказать биологический ответ молекул (столбец 'Activity') по их химическому составу (столбцы D1-D1776).

Предварительная обработка не требуется, данные уже закодированы и нормализованы.

В качестве метрики будем использовать F1-score.

Необходимо обучить две модели: логистическую регрессию и случайный лес. Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. Важно использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций не должно превышать 50.

## О данных

Задача основана на соревновании Kaggle: Predicting a Biological Response (Прогнозирование биологического ответа) - https://www.kaggle.com/c/bioresponse

Каждая строка представляет молекулу. 

Первый столбец Activity содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1]; 
Остальные столбцы D1-D1776 представляют собой молекулярные дескрипторы — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.



In [21]:
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn.model_selection import train_test_split #сплитование выборки

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
import optuna

%matplotlib inline
plt.style.use('seaborn')

In [2]:
data = pd.read_csv('data/train_sem09.csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3751 entries, 0 to 3750
Columns: 1777 entries, Activity to D1776
dtypes: float64(942), int64(835)
memory usage: 50.9 MB


## Строим модели

In [5]:
X = data.drop('Activity', axis=1)
y = data['Activity']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=1)


In [16]:
#Построим модель логистической регрессии и проверим её качество

lr = linear_model.LogisticRegression(max_iter=1000, random_state=1)
lr.fit(X_train,y_train)
y_test_pred = lr.predict(X_test)
print(metrics.f1_score(y_test_pred, y_test))

0.7808383233532934


In [17]:
#Построим модель случайного леса и проверим её качество

rf = ensemble.RandomForestClassifier(random_state=1)
rf.fit(X_train, y_train)
y_test_pred = rf.predict(X_test)
print(metrics.f1_score(y_test_pred, y_test))

0.8182912154031289


## Подбор гиперпараметров с помощью GridSearchCV

In [19]:
#Подбор параметров для модели логистической регрессии

param_grid = [
              {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регуляризации
              
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]

grid_search = GridSearchCV(estimator=linear_model.LogisticRegression(
                           random_state=1,
                           max_iter=1000), 
                           param_grid=param_grid, 
                           cv=5, 
                           n_jobs = -1
)  
%time grid_search.fit(X_train, y_train) 
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

Wall time: 13min 20s
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}


In [31]:
#Подбор параметров для модели случайного леса

param_grid = {'n_estimators':[100, 200, 300],
              'min_samples_leaf': [5, 10, 15, 20],
              'max_depth': [5, 10, 15, 20],
              'criterion':['entropy','gini']
              }

grid_search = GridSearchCV(estimator=ensemble.RandomForestClassifier(
                           random_state=1), 
                           param_grid=param_grid, 
                           cv=5, 
                           n_jobs = -1
)  
%time grid_search.fit(X_train, y_train) 
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

Wall time: 3min 15s
f1_score на тестовом наборе: 0.81
Наилучшие значения гиперпараметров: {'criterion': 'entropy', 'max_depth': 15, 'min_samples_leaf': 5, 'n_estimators': 200}


## Подбор гиперпараметров с помощью RandomizedSearchCV

In [27]:
#Подбор параметров для модели логистической регрессии

param_grid = [
              {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регуляризации
              
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]
            
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=1, max_iter=1000), 
    param_distributions=param_grid, 
    cv=5, 
    n_iter = 10, 
    n_jobs = -1
)  
%time random_search.fit(X_train, y_train) 
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

Wall time: 2min 24s
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'solver': 'saga', 'penalty': 'l2', 'C': 0.1}


In [30]:
#Подбор параметров для модели случайного леса

param_grid = {'n_estimators':[100, 200, 300],
              'min_samples_leaf': [5, 10, 15, 20],
              'max_depth': [5, 10, 15, 20],
              'criterion':['entropy','gini']
              }

random_search = RandomizedSearchCV(estimator=ensemble.RandomForestClassifier(random_state=1), 
                                   param_distributions=param_grid, 
                                   cv=5, 
                                   n_iter = 10, 
                                   n_jobs = -1
)

%time random_search.fit(X_train, y_train) 
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

Wall time: 27.8 s
f1_score на тестовом наборе: 0.81
Наилучшие значения гиперпараметров: {'n_estimators': 300, 'min_samples_leaf': 5, 'max_depth': 20, 'criterion': 'entropy'}


# Подбор гиперпараметров с помощью Hyperopt

In [91]:
#Для модели логистической регрессии без кроссвалидации

# зададим пространство поиска гиперпараметров
space = {'penalty': hp.choice('penalty', ['l1', 'l2']),
         'solver': hp.choice('solver', ['liblinear', 'saga']),
         'C': hp.choice('C', [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1])}

random_state = 1
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'penalty': str(params['penalty']), 
              'solver': str(params['solver']), 
              'C': float(params['C'])
              }
  
    # используем эту комбинацию для построения модели
    model = linear_model.LogisticRegression(**params, random_state=random_state, max_iter=2000)

    # обучаем модель
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    # score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [94]:
# начинаем подбор гиперпараметров

trials = Trials() # используется для логирования результатов. Они могут понадобиться для дальнейших поисков гиперпараметров.

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
         )
print("Наилучшие значения гиперпараметров {}".format(best))

 10%|████▊                                           | 2/20 [02:12<23:16, 77.58s/trial, best loss: -0.8791679412664423]

C:\Users\agner\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,



100%|███████████████████████████████████████████████| 20/20 [03:10<00:00,  9.52s/trial, best loss: -0.8856968215158926]
Наилучшие значения гиперпараметров {'C': 6, 'penalty': 1, 'solver': 0}


In [98]:
#Посчитаем метрику f1 для тестовой выборки с выбранными параметрами:

model = linear_model.LogisticRegression(penalty=best['penalty'],
                                        C=best['C'],
                                        solver=best['solver'],
                                        max_iter=2000, 
                                        random_state=1)
lr.fit(X_train,y_train)
y_test_pred = lr.predict(X_test)

print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.78


In [99]:
#Для модели логистической регрессии с кроссвалидацией

# зададим пространство поиска гиперпараметров
space = {'penalty': hp.choice('penalty', ['l1', 'l2']),
         'solver': hp.choice('solver', ['liblinear', 'saga']),
         'C': hp.choice('C', [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1])}

random_state = 1
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'penalty': str(params['penalty']), 
              'solver': str(params['solver']), 
              'C': float(params['C'])
              }
  
    # используем эту комбинацию для построения модели
    model = linear_model.LogisticRegression(**params, random_state=random_state, max_iter=2000)

    # обучаем модель
    model.fit(X, y)
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [100]:
# начинаем подбор гиперпараметров

trials = Trials() # используется для логирования результатов. Они могут понадобиться для дальнейших поисков гиперпараметров.

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
         )
print("Наилучшие значения гиперпараметров {}".format(best))

 45%|█████████████████████▌                          | 9/20 [08:32<12:04, 65.86s/trial, best loss: -0.7817633488644076]

C:\Users\agner\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,



 60%|████████████████████████████▏                  | 12/20 [12:50<08:06, 60.76s/trial, best loss: -0.7817633488644076]

C:\Users\agner\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,



 90%|██████████████████████████████████████████▎    | 18/20 [17:14<00:42, 21.40s/trial, best loss: -0.7819106289405601]

C:\Users\agner\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,



100%|███████████████████████████████████████████████| 20/20 [22:04<00:00, 66.21s/trial, best loss: -0.7819106289405601]
Наилучшие значения гиперпараметров {'C': 1, 'penalty': 1, 'solver': 0}


In [101]:
#Посчитаем метрику f1 для тестовой выборки с выбранными параметрами:

model = linear_model.LogisticRegression(penalty=best['penalty'],
                                        C=best['C'],
                                        solver=best['solver'],
                                        max_iter=2000, 
                                        random_state=1)
lr.fit(X_train,y_train)
y_test_pred = lr.predict(X_test)

print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.78


In [50]:
#Для модели случайного леса без кроссвалидации

# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 300, 100),
       'max_depth' : hp.quniform('max_depth', 5, 20, 5),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 5, 20, 5)
      }

random_state = 1
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    # score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [51]:
# начинаем подбор гиперпараметров
#%%time

trials = Trials() # используется для логирования результатов. Они могут понадобиться для дальнейших поисков гиперпараметров.

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|███████████████████████████████████████████████| 20/20 [00:43<00:00,  2.18s/trial, best loss: -0.9440538061754814]
Наилучшие значения гиперпараметров {'max_depth': 20.0, 'min_samples_leaf': 5.0, 'n_estimators': 100.0}


In [52]:
# рассчитаем метрику f1 для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.81


In [ ]:
#Для модели случайного леса с кроссвалидацией

# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 300, 100),
       'max_depth' : hp.quniform('max_depth', 5, 20, 5),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 5, 20, 5)
      }

random_state = 1
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    #score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [53]:
# начинаем подбор гиперпараметров
#%%time

trials = Trials() # используется для логирования результатов. Они могут понадобиться для дальнейших поисков гиперпараметров.

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|███████████████████████████████████████████████| 20/20 [00:38<00:00,  1.94s/trial, best loss: -0.9388753056234718]
Наилучшие значения гиперпараметров {'max_depth': 15.0, 'min_samples_leaf': 5.0, 'n_estimators': 200.0}


In [54]:
# рассчитаем метрику f1 для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82


# Подбор гиперпараметнов с помощью Optuna

In [113]:
# Для модели логистической регрессии

def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])
    C = trial.suggest_float('C', 0.01, 0.91, step=0.1)

    # создаем модель    
    model = linear_model.LogisticRegression(penalty=penalty,
                                            C=C,
                                            solver=solver,
                                            max_iter=2000, 
                                            random_state=1)    

    # обучаем модель
    model.fit(X_train, y_train)
    score = metrics.f1_score(y_train, model.predict(X_train))

    return score

In [114]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

[I 2022-09-14 16:02:17,896] A new study created in memory with name: LogisticRegression
[I 2022-09-14 16:02:39,418] Trial 0 finished with value: 0.8760330578512396 and parameters: {'penalty': 'l2', 'solver': 'saga', 'C': 0.6100000000000001}. Best is trial 0 with value: 0.8760330578512396.
[I 2022-09-14 16:02:39,667] Trial 1 finished with value: 0.816154309825196 and parameters: {'penalty': 'l1', 'solver': 'liblinear', 'C': 0.21000000000000002}. Best is trial 0 with value: 0.8760330578512396.
[I 2022-09-14 16:02:39,902] Trial 2 finished with value: 0.816154309825196 and parameters: {'penalty': 'l1', 'solver': 'liblinear', 'C': 0.21000000000000002}. Best is trial 0 with value: 0.8760330578512396.
[I 2022-09-14 16:04:31,547] Trial 3 finished with value: 0.8590604026845637 and parameters: {'penalty': 'l1', 'solver': 'saga', 'C': 0.7100000000000001}. Best is trial 0 with value: 0.8760330578512396.
[I 2022-09-14 16:04:31,910] Trial 4 finished with value: 0.8642878926501982 and parameters: {'

Wall time: 8min 32s


In [116]:
# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(**study.best_params,random_state=random_state, max_iter=1000)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.78


In [102]:
# Для модели случайного леса

def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 300, 100)
  max_depth = trial.suggest_int('max_depth', 5, 20, 5)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 5, 20, 5)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))

  return score

In [103]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

[I 2022-09-14 15:46:23,244] A new study created in memory with name: RandomForestClassifier
[I 2022-09-14 15:46:24,456] Trial 0 finished with value: 0.8475404827375497 and parameters: {'n_estimators': 100, 'max_depth': 10, 'min_samples_leaf': 20}. Best is trial 0 with value: 0.8475404827375497.
[I 2022-09-14 15:46:25,064] Trial 1 finished with value: 0.7996289424860854 and parameters: {'n_estimators': 100, 'max_depth': 5, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.8475404827375497.
[I 2022-09-14 15:46:28,407] Trial 2 finished with value: 0.8984709480122324 and parameters: {'n_estimators': 300, 'max_depth': 20, 'min_samples_leaf': 10}. Best is trial 2 with value: 0.8984709480122324.
[I 2022-09-14 15:46:30,413] Trial 3 finished with value: 0.8699112335475973 and parameters: {'n_estimators': 200, 'max_depth': 20, 'min_samples_leaf': 15}. Best is trial 2 with value: 0.8984709480122324.
[I 2022-09-14 15:46:33,610] Trial 4 finished with value: 0.9045566502463055 and parameters: {

Wall time: 43 s


In [104]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.81
